In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pywt

## This is for without baseline corretion data

In [2]:
# # data="/home/sultan/EEG_CP/CP_Extracted_CSV_Source_Dk_data/S1/Trig_1"
# dpath="/home/sultan/EEG_CP/CP_Extracted_CSV_Source_Dk_data/S1/Trig_1/S1Trig1_t1.csv"
# # dpath="/home/sultan/EEG_CP/CP_Extracted_CSV_Source_Dk_data/S1/Trig_1/S1Trig1_t99.csv"
# # dpath="/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S1/Trig_1/BC_S1Trig1_t1.csv"
# df1=pd.read_csv(dpath)
# tmp = df1.copy()
# tmp = tmp.drop(columns=['cc1'] )
# ## base line correction
# orgd=np.asmatrix(tmp)
# blm=np.mean(tmp.iloc[:,0:100],axis=1)
# bsm=np.asmatrix(blm).T
# ablc=orgd-bsm
# df2=np.asarray(ablc)
# # df2[1]
# df2.shape

## This is for baseline correction data

In [3]:
# # data="/home/sultan/EEG_CP/CP_Extracted_CSV_Source_Dk_data/S1/Trig_1"
# # dpath="/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S1/Trig_1/BC_S1Trig1_t1.csv"
# df1=pd.read_csv(dpath)
# tmp = df1.copy()
# tmp = tmp.drop(columns=['cc1'] )
# # orgd=np.asmatrix(tmp)
# # df2=orgd
# # df2.shape
# df2=tmp.iloc[:,1:]
# df2=np.asarray(df2)
# df2
# data=df2

In [4]:
# # # data="/home/sultan/EEG_CP/CP_Extracted_CSV_Source_Dk_data/S1/Trig_1"
# dpath="/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S1/Trig_1/BC_S1Trig1_t1.csv"
# df1=pd.read_csv(dpath)
# tmp = df1.copy()
# tmp = tmp.drop(columns=['cc1'] )
# # orgd=np.asmatrix(tmp)
# # df2=orgd
# # df2.shape
# df2=tmp.iloc[:,1:]
# df2=np.asarray(df2)
# df2
# data=df2

### Frequency with scale
## theta=5-8, alpha=9-13, Beta=14-29,gamma=30> 

In [5]:
dt =1/500.0  # 100 Hz sample
frequencies = pywt.scale2frequency('morl', np.arange(1,101))/dt
thein=np.logical_and(frequencies>=5, frequencies<=8)
theindex=np.where(thein)
alpin=np.logical_and(frequencies>=9, frequencies<=13)
alpindex=np.where(alpin)
betin=np.logical_and(frequencies>=14, frequencies<=29)
betindex=np.where(betin)
gamin=np.logical_and(frequencies>=30, frequencies<=100)
gamindex=np.where(gamin)

## Spectral_single trial

In [6]:
# plt.figure(8,10)
# f, ax = plt.subplots(8,10,figsize=(20,15))
def spectral_single(theindex,alpindex,betindex,gamindex,df2):
    t=np.linspace(-200,800, 500, endpoint=False)
    widths = np.arange(1,100)
    te=[];al=[];be=[];ga=[];
    for i in range(0,68):
        cwtmatr, freqs = pywt.cwt(df2[i], widths, 'morl')
        theta=np.mean(cwtmatr[theindex,:][0],axis=0)
        alpha=np.mean(cwtmatr[alpindex,:][0],axis=0)
        beta=np.mean(cwtmatr[betindex,:][0],axis=0)
        gamma=np.mean(cwtmatr[gamindex,:][0],axis=0)
        te.append(theta);al.append(alpha);be.append(beta);ga.append(gamma)
    #     ax=plt.subplot(8,9,i+1)
    #     ax.imshow(cwtmatr, extent=[-200, 800, 1, 100], cmap='jet', aspect='auto',vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max()) # doctest:
    #     plt.title(i)
    # plt.show() # doctest: +SKIP
    return te,al,be,ga

In [7]:
def evoked_induced(mychunk):
    alld=tz=az=bz=gz= np.zeros((68, 500))
    cl=len(mychunk)
    for c in range(0,cl):
    #     print mychunk[c]
        df1=pd.read_csv(mychunk[c])
        tmp = df1.copy()
        tmp = tmp.drop(columns=['cc1'] )
        # orgd=np.asmatrix(tmp)
        # df2=orgd
        # df2.shape
        df2=tmp.iloc[:,1:]
        df2=np.asarray(df2);
        alld=alld+df2
        t,a,b,g=spectral_single(theindex,alpindex,betindex,gamindex,df2)
        tz=tz+abs(np.asarray(t));az=az+abs(np.asarray(a));bz=bz+abs(np.asarray(b));gz=gz+abs(np.asarray(g));
    alld=alld/cl    
    ert,era,erb,erg=spectral_single(theindex,alpindex,betindex,gamindex,alld)
    indt=tz/cl-ert;inda=az/cl-era;indb=bz/cl-erb;indg=gz/cl-erg 
    tevk=np.mean(ert,axis=1);aevk=np.mean(era,axis=1);bevk=np.mean(erb,axis=1); gevk=np.mean(erg,axis=1);
    teind=np.mean(indt,axis=1);alind=np.mean(inda,axis=1); beind=np.mean(indb,axis=1);gaind=np.mean(indg,axis=1);
    evkoed=np.concatenate((tevk,aevk,bevk,gevk),axis=0);
    induced=np.concatenate((teind,alind,beind,gaind),axis=0);
    return evkoed,induced; ## combined all frequency bands
#     return tevk,aevk,bevk,gevk,teind,alind,beind,gaind
#     return ert,era,erb,erg, indt,inda,indb,indg; ## This line for return wavelet transform value 

### Features extracting accrording to Frequency band 

In [8]:
## number of sample and repeated trials:
sample=100; ntr=30;
dirpath="/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/"
# # Remove the existence folder with files 
# shutil.rmtree(despath)
# # subject and Triger
# sub=3;# subject+1
# Ev=6;#event+1
evkalls=[];
indalls=[];
c=0;
sub=range(1,51); ##sub+1
Ev=[1,5];
for k in range(0,len(sub)):
    for j in range(0,len(Ev)):
        b=dirpath+'S'+str(sub[k])+'/'+'Trig_'+str(Ev[j])+'/'
        dpin=b
        print dpin
        from os import walk
        from os.path import isfile, join
        f = []
        for (dpin, dirnames, filenames) in walk(dpin):
            f.extend(join(dpin, x) for x in filenames if x.endswith(".csv"))
            break
#         print f
#         sample=100;# How many samples need to average out
#         totalck =len(f)*1.0 / sample 
#         totalchunk=int(math.ceil(totalck))
#         totalchunk = len(f)/sample 
#         print (totalchunk)
#         descr=despath+'S'+str(sub[k])+'/'+'Trig_'+str(Ev[j]); # Make directory 
#         os.makedirs(descr)
#         import numpy as np
        for i in range(ntr): # we will generate two at a time
            mychunk = np.random.choice(f,size=sample,replace=False)
            #print mychunk
            # make sure the chunk element does not exist in the original list 
#             f = list(set(f) - set(mychunk))
            #print f
            #fname=descr+'Av'+str(i)
            #fname='S'+str(k)+'event'+str(j)+'av'+str(i)
#             fname=despath+'S'+str(sub[k])+'/'+'Trig_'+str(Ev[j])+'/'+'S'+str(sub[k])+'Trig'+str(Ev[j])+'_'+str(sample)+'av'+str(i)
#             print i
#             print('S'+str(sub[k])+':',c)
#             c=c+1
            a,b=evoked_induced(mychunk);
            a1=pd.DataFrame(a)
            a2=a1.T
            b1=pd.DataFrame(b)
            b2=b1.T
            evkalls.append(a2.values[0])
            indalls.append(b2.values[0])
# print (evkalls)
aevoked=pd.DataFrame(evkalls)  
ainducd=pd.DataFrame(indalls) 

/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S1/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S1/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S2/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S2/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S3/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S3/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S4/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S4/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S5/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S5/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S6/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S6/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S7/Trig_1/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S7/Trig_5/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S8/Trig_1/
/home/sultan/EEG_CP/Basel

## only Tk3

In [9]:
dirpath="/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/"
# # Remove the existence folder with files 
# shutil.rmtree(despath)
# # subject and Triger
# sub=3;# subject+1
# Ev=6;#event+1
# sample=75;
evkallstk3=[];
indallstk3=[];
c=0;
sub=range(1,51); ##sub+1
Ev=[3];
for k in range(0,len(sub)):
    for j in range(0,len(Ev)):
        b=dirpath+'S'+str(sub[k])+'/'+'Trig_'+str(Ev[j])+'/'
        dpin=b
        print dpin
        from os import walk
        from os.path import isfile, join
        f = []
        for (dpin, dirnames, filenames) in walk(dpin):
            f.extend(join(dpin, x) for x in filenames if x.endswith(".csv"))
            break
#         print f
#         sample=100;# How many samples need to average out
#         totalck =len(f)*1.0 / sample 
#         totalchunk=int(math.ceil(totalck))
#         totalchunk = len(f)/sample 
#         print (totalchunk)
#         descr=despath+'S'+str(sub[k])+'/'+'Trig_'+str(Ev[j]); # Make directory 
#         os.makedirs(descr)
#         import numpy as np
        for i in range(ntr): # we will generate two at a time
            mychunk = np.random.choice(f,size=sample,replace=False)
            #print mychunk
            # make sure the chunk element does not exist in the original list 
#             f = list(set(f) - set(mychunk))
            #print f
            #fname=descr+'Av'+str(i)
            #fname='S'+str(k)+'event'+str(j)+'av'+str(i)
#             fname=despath+'S'+str(sub[k])+'/'+'Trig_'+str(Ev[j])+'/'+'S'+str(sub[k])+'Trig'+str(Ev[j])+'_'+str(sample)+'av'+str(i)
#             print len(f)
#             print('S'+str(sub[k])+':',c)
#             c=c+1
            a,b=evoked_induced(mychunk);
            a1=pd.DataFrame(a)
            a2=a1.T
            b1=pd.DataFrame(b)
            b2=b1.T
            evkallstk3.append(a2.values[0])
            indallstk3.append(b2.values[0])
# print (evkalls)
aevokedtk3=pd.DataFrame(evkallstk3)  
ainducdtk3=pd.DataFrame(indallstk3) 

/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S1/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S2/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S3/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S4/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S5/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S6/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S7/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S8/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S9/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S10/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S11/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S12/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S13/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S14/Trig_3/
/home/sultan/EEG_CP/Baseline_Corrected_Source_CP_Data/S15/Trig_3/
/home/sultan/EEG_CP

In [10]:
# aevoked[:100]

# Stop for writing

In [11]:
# stop check 

## Writing label and csv

In [12]:
# aevoked.to_csv('Evoked_tk15_all.csv')
# ainducd.to_csv('Induced_tk15_all.csv')
# ainducdtk3.to_csv('Induceded_tk3_all.csv')
# aevokedtk3.to_csv('Evoked_tk3_all.csv')

## Evoked

In [13]:
# wrpath="/"

In [14]:
aevoked.insert(0,'label',np.ones(len(aevoked)), True)
aevokedtk3.insert(0,'label',np.zeros(len(aevokedtk3)), True)

In [15]:
dallevoked=pd.concat([aevoked,aevokedtk3])
dallevoked.to_csv('Evoked_'+str(sample)+'_Tk153.csv')

### Induced 

In [16]:
ainducd.insert(0,'label',np.ones(len(ainducd)), True)
ainducdtk3.insert(0,'label',np.zeros(len(ainducdtk3)), True)

In [17]:
dallinduced=pd.concat([ainducd,ainducdtk3])
dallinduced.to_csv('Induced_'+str(sample)+'_Tk153.csv')

In [18]:
# dallinduced.head(3)

In [19]:
# tt,a,b,g=spectral_single(theindex,alpindex,betindex,gamindex,data)